In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn import svm
import pandas as pd
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, confusion_matrix, ConfusionMatrixDisplay
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
def data_frame_adder(temp_frame,file_name):
    file_name=file_name.split('.csv')[0]
    temp_frame['file_name']=file_name
    temp_frame['subject']=temp_frame['file_name'].apply(lambda x: int(x.split('subject_')[1].split('_')[0]))
    temp_frame['session']=temp_frame['file_name'].apply(lambda x: int(x.split('subject_')[1].split('_')[1]))
    #temp_frame['type']=temp_frame['file_name'].apply(lambda x: x.split('__')[1])
    temp_frame = temp_frame.drop(columns=['file_name'])
    temp_frame = temp_frame[temp_frame.columns[::-1]]
    temp_frame = temp_frame.sort_values(by=['session','subject'], axis=0)
    col_list1 = list(temp_frame.columns)[0:2]
    col_list2 = list(temp_frame.columns)[2:]
    col_list2.sort()
    col_list=col_list1+col_list2
    
    #temp_frame = swap_columns(temp_frame, 'subject', 'session')
    # Swapping the columns
    temp_frame = temp_frame.reindex(columns=col_list)

    return temp_frame
Training_data_files=glob.glob('TestData/*.csv')
file_name_list_x=[]
file_name_list_x_time=[]
file_name_list_y=[]
file_name_list_y_time=[]
for file_name in tqdm(Training_data_files):
    if 'x.csv' in file_name:
        file_name_list_x.append(file_name)
    elif 'x_time.csv' in file_name:
        file_name_list_x_time.append(file_name)
    elif 'y.csv' in file_name:
        file_name_list_y.append(file_name)
    else:
        file_name_list_y_time.append(file_name)
file_name_list_x.sort()
file_name_list_x_time.sort()
file_name_list_y.sort()
file_name_list_y_time.sort()
Subject_list= [f.split('_')[1] for f in file_name_list_x]
Session_list= [f.split('_')[2] for f in file_name_list_x]

x_sensor_dat=pd.DataFrame([])
y_sensor_dat=pd.DataFrame([])
x_time_dat=[]
for sub,sess in tqdm(zip(Subject_list,Session_list)):
    #x_sensor_dat=pd.DataFrame(x_sensor_dat.append(pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x.csv",header=None)))
    
    x_sensor_dat=pd.concat([pd.read_csv(f"TestData/subject_{sub}_{sess}__x.csv",header=None),pd.read_csv(f"TestData/subject_{sub}_{sess}__x_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1)
    features_to_normalize = [0,1,2,3,4,5]
    features_to_normalize = [0,1,2,3,4,5]
    scaler = StandardScaler()
    scaler.fit(x_sensor_dat[features_to_normalize])
    scaled = scaler.fit_transform(x_sensor_dat[features_to_normalize])
    scaled_df = pd.DataFrame(scaled, columns=x_sensor_dat[features_to_normalize].columns)
    x_sensor_dat[features_to_normalize]=scaled_df

#    x_sensor_dat[features_to_normalize] = x_sensor_dat[features_to_normalize].apply(lambda x:(x-x.mean())/ x.std(), axis=0)
    #apply(lambda x:(x-x.min())/(x.max()-x.min()))
    x_sensor_dat['measurements']=x_sensor_dat.apply(lambda x: [x[0],x[1],x[2],x[3],x[4],x[5]],axis=1)
    x_sensor_dat=x_sensor_dat[['time_stamp','measurements']]
    #x_sensor_dat=x_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    x_sensor_dat['Subject']=int(sub)
    x_sensor_dat['Session']=int(sess)
    #x_sensor_dat=x_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    
    y_sensor_dat=pd.concat([pd.read_csv(f"TestData/subject_{sub}_{sess}__y.csv",header=None),pd.read_csv(f"TestData/subject_{sub}_{sess}__y_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1).rename(columns={0:'labels'})
    #y_sensor_dat['labels']=y_sensor_dat.apply(lambda y: [y[0],y[1],y[2],y[3]],axis=1)
    y_sensor_dat=y_sensor_dat[['time_stamp','labels']]
    #y_sensor_dat["time_stamp"]=pd.to_datetime(y_sensor_dat["time_stamp"],unit='s').round('ms')
    trp=pd.concat(
        [pd.DataFrame({'labels':0},index=[x_sensor_dat.index[-1]]),pd.DataFrame({'time_stamp':np.max(x_sensor_dat['time_stamp'])},index=[x_sensor_dat.index[-1]])
                         ]
         ,axis=1)
    y_sensor_dat=pd.concat([y_sensor_dat,trp],axis=0)
    trp=pd.concat([pd.DataFrame({'labels':0},index=[x_sensor_dat.index[0]]),pd.DataFrame({'time_stamp':np.min(x_sensor_dat['time_stamp'])},index=[x_sensor_dat.index[0]])],axis=1)
    y_sensor_dat=pd.concat([trp,y_sensor_dat],axis=0)
    # y_sensor_dat=y_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    #y_sensor_dat['Subject']=int(sub)
    #y_sensor_dat['Session']=int(sess)
    x_sensor_dat["time_stamp"]=pd.to_datetime(x_sensor_dat["time_stamp"],unit='s').round('ms')
    x_sensor_dat=x_sensor_dat.set_index('time_stamp')
    x_sensor_dat=x_sensor_dat.asfreq('0.025S')
    
    y_sensor_dat["time_stamp"]=pd.to_datetime(y_sensor_dat["time_stamp"],unit='s').round('ms')
    y_sensor_dat=y_sensor_dat.set_index('time_stamp')
    y_sensor_dat=y_sensor_dat.asfreq(freq='0.025S', method='bfill')
    x_y_dat=pd.merge(x_sensor_dat,y_sensor_dat, how='inner', left_index=True, right_index=True)
    x_y_dat.to_parquet(f"pre_processed_testing/subject_{sub}_session_{sess}.gzip",compression='gzip')
files=glob.glob('pre_processed_testing/*.gzip')
total_training_data=pd.DataFrame([])
for f in tqdm(files):
    total_training_data=pd.concat([total_training_data,pd.read_parquet(f, engine='auto')],axis=0)
total_training_data=total_training_data.sort_values(['Subject','Session','time_stamp'])
total_training_data=total_training_data.reset_index()
total_training_data['epoch'] = total_training_data['time_stamp'].sub(pd.Timestamp('1970-01-01 00:00:00.000'))
total_training_data['epoch']=total_training_data['epoch'].dt.total_seconds()
total_training_data=total_training_data[['Subject','Session','epoch','measurements','labels']]
total_training_data_grouped=total_training_data.groupby(['Subject','Session','epoch']).agg({'measurements':[np.mean],'labels':[np.mean]})
total_training_data_grouped.columns = total_training_data_grouped.columns.droplevel(1)
total_training_data_grouped=total_training_data_grouped.reset_index(level=['epoch'])
total_training_data_grouped.to_parquet(f"combined_sampled/testing.gzip",compression='gzip')
testing_combined=pd.read_parquet('combined_sampled/testing.gzip', engine='auto')
testing_combined=testing_combined.reset_index()
testing_combined=testing_combined[['measurements','labels']]
X_test=np.vstack(testing_combined['measurements']).reshape(-1,3,2,1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 262144.00it/s]
4it [00:02,  1.88it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 37.23it/s]


In [15]:
num_classes=4

In [19]:
def make_model(input_shape,num_classes=num_classes):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv2D(filters=64, kernel_size=2, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv2D(filters=64, kernel_size=2, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv2D(filters=64, kernel_size=2, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling2D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="relu")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=X_test.shape[1:],num_classes=num_classes)
# epochs = 500
# batch_size = 32

# callbacks = [
#     keras.callbacks.ModelCheckpoint(
#         "best_model.h5", save_best_only=True, monitor="val_loss"
#     ),
#     keras.callbacks.ReduceLROnPlateau(
#         monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
#     ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
# ]
# model.compile(
#     optimizer="adam",
#     loss="categorical_crossentropy",
#     metrics=["accuracy"],
# )

Metal device set to: Apple M1


2023-04-17 22:56:06.167046: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-17 22:56:06.167458: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

,measurements,labels
0,"[-0.03765202030266226, 0.09368572626400568, -1...",0.0
1,"[-0.03524098598251466, 0.09306726091291935, -1...",NaN
2,"[-0.03380178732749251, 0.0932526808924517, -1....",NaN
3,"[-0.031089793317064792, 0.0952962881937525, -1...",NaN
4,"[-0.025079759587105822, 0.09672302515454348, -...",NaN
...,...,...
184147,"[0.159406356135594, 0.16277825502985702, -0.47...",NaN
184148,"[0.16483386876811393, 0.1645012814504027, -0.4...",NaN
184149,"[0.17994221577331065, 0.15910908643511365, -0....",NaN
184150,"[0.17649126596710446, 0.16459274505127874, -0....",0.0


In [ ]:
loaded_model.load_weights("model.h5")
print("Loaded model from disk")